[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Optimization Methods

## Convex Optimization - Constraint Optimization - Least Squares with Equality Constraints

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.001 | 03/03/2024 | Royi Avital | Added remarks on the model matrix and values vector                |
| 1.0.000 | 09/02/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0012LinearFitL1.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Optimization
import cvxpy as cp

# Miscellaneous
import os
import math
from platform import python_version
import random

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
from matplotlib.colors import LogNorm, Normalize, PowerNorm
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
%matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions


In [ ]:
# Parameters

# Data
numCoeff    = 11
numSamples  = 110
noiseStd    = 0.005
convShape   = CONVOLUTION_SHAPE_VALID


# Solver
stepSizeMode    = STEP_SIZE_MODE_ADAPTIVE
stepSize        = 0.001
numIterations   = 2500

# Verification
ε      = 1e-6 #<! Error threshold

## Linear Fit with Equality Constraints

The _Linear Fit_ / _Least Squares_ with Equality Constraints is given by:

$$\begin{align}
\arg \min_{\boldsymbol{x}} \quad & {\left\| \boldsymbol{A} \boldsymbol{x} - \boldsymbol{b} \right\|}_{2}^{2} \\
\text{subject to} \quad & \begin{aligned} 
\boldsymbol{C} \boldsymbol{x} & = \boldsymbol{d} \\
\end{aligned}
\end{align}$$

## Least Squares with Sum Constraints

An _High Pass Filter_ (HPF) is a filter decays low frequencies.  
A simplistic way to force an HPF is to force it will vanish the DC component.  
This can be done by forcing the sum of its coefficients to be zero.

This notebook deals with estimating an High Pass Filter (HPF) using Projected Gradient Descent.
The problems equivalent to a Least Squares problem with a linear equality constraint.

This notebook shows how to solve the problem:

$$\begin{align}
\arg \min_{\boldsymbol{x}} \quad & {\left\| \boldsymbol{A} \boldsymbol{x} - \boldsymbol{b} \right\|}_{2}^{2} \\
\text{subject to} \quad & \begin{aligned} 
\sum {x}_{i} & = 0 \\
\end{aligned}
\end{align}$$

The notebook:

 - Calculates a solution using DCP Solver (Reference).
 - Calculates a solution using _Projected Gradient Descent_.

* <font color='brown'>(**#**)</font> Constraints are useful for undetermined case where the number of solutions is infinite.
* <font color='red'>(**?**)</font> Formulate the constraint as a dot product (Vector).


## Generate Data


The data model is a stream of samples going through an LTI system.  
In our case the LTI system is built by an HPF filter:

$$ \boldsymbol{y} = \operatorname{conv} \left( \boldsymbol{x}, \boldsymbol{h} \right) + \boldsymbol{n} $$

Where:
 - $\boldsymbol{x}$ - The data samples.
 - $\boldsymbol{h}$ - The filter coefficients.
 - $\boldsymbol{n}$ - The white noise samples (AWGN).

Since the model is linear is can be written in a matrix form:

$$ \boldsymbol{y} = \boldsymbol{X} \boldsymbol{h} + \boldsymbol{n} $$

* <font color='brown'>(**#**)</font> Since the model is a convolution (LTI) system, the matrix $\boldsymbol{X}$ is a [Toeplitz Matrix](https://en.wikipedia.org/wiki/Toeplitz_matrix).
* <font color='brown'>(**#**)</font> Read on [`numpy.convolve()`](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html) and [`scipy.signal.convolve()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html). Pay attention to the `method` option in [`scipy.signal.convolve()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html).

In [ ]:
# Generate / Load the Data

vG = np.random.rand(numSamples + numOutliers)
vB = ((modelSlope * vG) + modelIntercept) + (noiseStd * np.random.rand(numSamples + numOutliers))
vI = np.random.choice(numSamples + numOutliers, numOutliers, replace = False) #<! Choose `numOutliers` of the points
vB[vI] += (outlierStd * np.random.randn(numOutliers)) #<! The values vector

mA = np.column_stack((np.ones_like(vG), vG)) #!< The model matrix

In [ ]:
# Display the Data

hF, hA = plt.subplots(figsize = (16, 8))
hA.plot(vG, modelSlope * vG + modelIntercept, label = 'Model')
hA.scatter(vG, vB, s = 50, color = lMatPltLibclr[1], label = 'Data Samples')
hA.set_xlabel(r'$x$')
hA.set_ylabel(r'$y$')
hA.set_title('Data Samples')

hA.legend();

## Linear Fit

In this section 3 linear fits will be calculated:

1. The _Least Squares_ (Squared ${L}^{2}$ Norm).
2. The ${L}^{1}$ _Linear Fit_ using a direct formulation with a DCP solver.
3. The ${L}^{1}$ _Linear Fit_ using a _Linear Programming_ formulation with a DCP solver.

* <font color='brown'>(**#**)</font> The ${L}^{1}$ solutions will be solved by a DCP solver: `CVXPY`.


In [ ]:
# Least Squares Fit

#===========================Fill This===========================#
# 1. Estimate the model parameters using Least Squares.
# !! You may find `np.linalg.lstsq` useful.

vXLS, *_ = ???

#===============================================================#

* <font color='red'>(**?**)</font> How many components in `vXLS`?  
Map the elements of `vXLS` into the model parameters.

In [ ]:
# L1 Linear Fit
# This is the reference solution.

vXL1Ref = cp.Variable(2)

cpObjFun = cp.Minimize(cp.norm(mA @ vXL1Ref - vB, 1)) #<! Objective Function
oCvxPrb = cp.Problem(cpObjFun)
oCvxPrb.solve(solver = cp.CLARABEL)

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

vXL1Ref = vXL1Ref.value

In [ ]:
# L1 Linear Fit
# This is the solution using Linear Programming formulation.
# It will be solved using `CVXPY`.

#===========================Fill This===========================#
# 1. Create the auxiliary variable `vT`.
# 1. Define the objective function.
# 3. Define the constraints.
# 4. Solve the problem using `CVXPY`.
# !! You may use list operations to define constraints.

vT   = cp.Variable(???)
vXL1 = cp.Variable(???)

cpObjFun = ??? #<! Objective Function
cpConst = ??? #<! Constraints
oCvxPrb = ??? #<! Problem
#===============================================================#

oCvxPrb.solve(solver = cp.CLARABEL)

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

vXL1 = vXL1.value

In [ ]:
# Display Results

hF, hA = plt.subplots(figsize = (16, 8))
hA.plot(vG, modelSlope * vG + modelIntercept, label = 'Model')
hA.scatter(vG, vB, s = 50, color = lMatPltLibclr[-1], label = 'Data Samples')
hA.plot(vG, vXLS[1] * vG + vXLS[0], label = 'Least Squares')
hA.plot(vG, vXL1Ref[1] * vG + vXL1Ref[0], label = 'L1 Fit Reference')
hA.plot(vG, vXL1[1] * vG + vXL1[0], label = 'L1 Fit')
hA.set_xlabel(r'$x$')
hA.set_ylabel(r'$y$')
hA.set_title('Fitted Models')

hA.legend();

* <font color='red'>(**?**)</font> Looking at the results, explain the name "Robust Regression".
* <font color='red'>(**?**)</font> Why is the _Least Squares_ the more common method?